In [ ]:
import zipfile
import os
import pandas as pd
from hume import HumeBatchClient
from hume.models.config import FaceConfig

In [ ]:
# A trial for 15 images

# Initialize the Hume.ai client
client = HumeBatchClient("9FfD6nMezmjb1hvTaWwy09YGBCWDwL9NuK7cFipfZjq49iW7")

# Set the path for local images
image_dir = "./train"
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Limit to only analyze the first 15 images
image_files = image_files[:15]

# Create an empty DataFrame to store results
results_df = pd.DataFrame()

# Submit analysis job
config = FaceConfig()  # Initialize analysis configuration
job = client.submit_job(None, [config], files=image_files)
print("Running facial analysis...")
details = job.await_complete()

# Download analysis results
artifacts_path = os.path.join(image_dir, 'artifacts.zip')
job.download_artifacts(artifacts_path)

# Extract analysis results
with zipfile.ZipFile(artifacts_path, 'r') as zip_file:
    zip_file.extractall(os.path.join(image_dir, 'predictions'))

# Set the path for the extracted directory
predictions_path = os.path.join(image_dir, 'predictions')

# Create an empty DataFrame to store results
results_df = pd.DataFrame()

# Recursively traverse the predictions directory to find all CSV files
for root, dirs, files in os.walk(predictions_path):
    for file in files:
        if file.endswith('.csv'):
            csv_path = os.path.join(root, file)
            batch_results = pd.read_csv(csv_path)
            results_df = pd.concat([results_df, batch_results], ignore_index=True)

# Save the consolidated results to a CSV
results_csv_path = '/Users/liuguanhong/Desktop/facial_analysis_results.csv'
results_df.to_csv(results_csv_path, index=False)
print(f"Facial analysis results saved to '{results_csv_path}'.")

In [ ]:
# For all images

# Initialize the Hume.ai client
client = HumeBatchClient("9FfD6nMezmjb1hvTaWwy09YGBCWDwL9NuK7cFipfZjq49iW7")

# Set the path for local images
image_dir = "./train"
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Create an empty DataFrame to store results
results_df = pd.DataFrame()

# Batch process images to avoid memory issues
batch_size = 10  # Adjust this size based on your system resources
for i in range(0, len(image_files), batch_size):
    batch_files = image_files[i:i+batch_size]

    # Submit analysis job
    config = FaceConfig()  # Initialize analysis configuration
    job = client.submit_job(None, [config], files=batch_files)
    print("Running facial analysis...")
    details = job.await_complete()

    # Download analysis results
    artifacts_path = os.path.join(image_dir, f'artifacts_batch_{i//batch_size}.zip')
    job.download_artifacts(artifacts_path)

    # Extract analysis results
    with zipfile.ZipFile(artifacts_path, 'r') as zip_file:
        zip_file.extractall(os.path.join(image_dir, f'predictions_batch_{i//batch_size}'))

    # Load analysis results into DataFrame
    predictions_path = os.path.join(image_dir, f'predictions_batch_{i//batch_size}')
    for csv_file in os.listdir(predictions_path):
        if csv_file.endswith('.csv'):
            batch_results = pd.read_csv(os.path.join(predictions_path, csv_file))
            results_df = pd.concat([results_df, batch_results], ignore_index=True)

# Save the consolidated results to CSV
results_df.to_csv('/Users/liuguanhong/Desktop/facial_analysis_results.csv', index=False)
print("Facial analysis results saved to 'facial_analysis_results.csv'.")